In [0]:
import pandas as pd 
import json
import math 
from pandas import json_normalize
import numpy as np 
from datetime import datetime 
from pyspark.sql import SparkSession

#storage_account_name = "wearabilitiesdatalake"
#storage_account_access_key = "KJwuJeolHGFTaCxOtJ85OczQuGaqd5ZX7KGwaZJuv3RbETBcZDuqds4H577Ll5UOlUNIPcU1joC7EZF8QU8InA=="
#file_location = "https://wearabilitiesdatalake.blob.core.windows.net/wearabilitiesfilesystem/data/dbo_User.csv"
#file_type = "csv"
#spark.conf.set(
#  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
#  storage_account_access_key)
#user_df = spark.read.format(file_type).option("inferSchema", "true").load(file_location)

user_df = spark.read.csv("/mnt/datalakegen2/data/dbo_User.csv",inferSchema="True",header="True").toPandas()
deliverycue_df = spark.read.csv("/mnt/datalakegen2/data/DeliveryCue.csv",inferSchema="True",header="True").toPandas()
deliverycue_splittedbyelement_df = spark.read.csv("/mnt/datalakegen2/Cont1/Splitted_byelemtns_devcue-00001.csv",inferSchema="True",header="True").toPandas()
#category_df = spark.read.csv("/mnt/datalakegen2/data/dbo_Category.csv",inferSchema="True",header="True").toPandas()
country_df = spark.read.csv("/mnt/datalakegen2/data/dbo_Country.csv",inferSchema="True",header="True").toPandas()
diagnosis_df = spark.read.csv("/mnt/datalakegen2/data/dbo_Diagnosis.csv",inferSchema="True",header="True").toPandas()
#files_df = spark.read.csv("/mnt/datalakegen2/data/dbo_Files.csv",inferSchema="True",header="True").toPandas()
#gender_df = spark.read.csv("/mnt/datalakegen2/data/dbo_Gender.csv",inferSchema="True",header="True").toPandas()
individualdiagnosis_df = spark.read.csv("/mnt/datalakegen2/data/dbo_IndividualDiagnosis.csv",inferSchema="True",header="True").toPandas()
organizationrole_df = spark.read.csv("/mnt/datalakegen2/data/dbo_OrganizationRole.csv",inferSchema="True",header="True").toPandas()
#race_df = spark.read.csv("/mnt/datalakegen2/data/dbo_Race.csv",inferSchema="True",header="True").toPandas()
#refreshtoken_df = spark.read.csv("/mnt/datalakegen2/data/dbo_RefreshToken.csv",inferSchema="True",header="True").toPandas()
relationship_df = spark.read.csv("/mnt/datalakegen2/data/dbo_Relationship.csv",inferSchema="True",header="True").toPandas()
role_df = spark.read.csv("/mnt/datalakegen2/data/dbo_Role.csv",inferSchema="True",header="True").toPandas()
user_device = spark.read.csv("/mnt/datalakegen2/data/dbo_UserDevice.csv",inferSchema="True",header="True").toPandas()
deliverycue_splittedbydevid_df = spark.read.csv("/mnt/datalakegen2/Cont1/CaregiverDevCue_DEVIDsplitted-00001.csv",inferSchema="True",header="True").toPandas()
individual_diagnosis_df = spark.read.csv("/mnt/datalakegen2/data/dbo_IndividualDiagnosis.csv",inferSchema="True",header="True").toPandas()
scenes = spark.read.csv("/mnt/datalakegen2/Cont1/Scenes_devcue-00001.csv",inferSchema="True",header="True").toPandas()


/databricks/spark/python/pyspark/sql/pandas/utils.py:91: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [Phone] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
  warnings.warn(


### CAREGIVER DATASETS GENERATION

In [0]:
caregiver_patient = user_df.groupby(['IdParent'])['IdParent'].count()
caregiver_patient_df = pd.DataFrame(caregiver_patient)
caregiver_patient_df.rename(columns = {'IdParent':'Individuals'}, inplace = True)
caregiver_patient_df.reset_index(inplace=True)
caregiver_patient_df.rename(columns = {'IdParent':'CaregiverID'}, inplace = True)
caregiver_id = user_df[user_df['IdRole'] == 3]['Id']
cgv_id = [int(item) for item in caregiver_id]
organization_id = user_df[user_df['IdRole'] == 4]['Id']

In [0]:
import ast 
from ast import literal_eval
def ___convert_to_ts(d):
  return datetime.strptime(np.datetime_as_string(d,unit='s'), '%Y-%m-%dT%H:%M:%S').timestamp()
def ___convert_to_datetime(d):
  return datetime.strptime(np.datetime_as_string(d,unit='s'), '%Y-%m-%dT%H:%M:%S')

caregiver_devcuetimerecord = pd.DataFrame(columns=['CaregiverID','DailyRecord','WeeklyRecord','MonthlyRecord','HistoricalRecord','AssignedUserCount','ElementUsage','LatestCueDate','TotalizedClassification','TotalizedType'])
deliverycue_table_temp = pd.DataFrame(columns = ['CaregiverID','DevCueID','DeviceID','IndividualsData','ST_total_devcue','AT_total_devcue'])
userdata_by_devcue = pd.DataFrame();
DELIVERYCUE_FINAL_TAB = pd.DataFrame();
#deliverycue_table_temp=[];
scenes_df = scenes.assign(DeviceId=scenes.DeviceId.str.split("|")).explode('DeviceId')
scenes_df = scenes_df.reset_index();
caregiver_deliverycue_count = scenes_df.groupby(['PartitionKey'])['PartitionKey'].count()
caregiver_deliverycue_count_df = pd.DataFrame(caregiver_deliverycue_count)
caregiver_deliverycue_count_df.rename(columns = {'PartitionKey':'DeliveryCueCount'}, inplace = True)
caregiver_deliverycue_count_df.reset_index(inplace=True)
caregiver_deliverycue_count_df.rename(columns = {'PartitionKey':'UserID'}, inplace = True)
scene_data = pd.DataFrame(columns=['CaregiverID','SceneID','SceneType','AnswerType'])
for index in scenes_df.index:
        key = scenes_df['RowKey'][index];
        caregiver_id = scenes_df['PartitionKey'][index];
        title = scenes_df['Title'][index];
        date = scenes_df['CreationDate'][index];
        device = scenes_df['DeviceId'][index] 
        temp_temp = scenes_df['Scenes'][index];#scenes_df[scenes_df['RowKey'] == key]['DeviceId'].item()
        user_list = pd.DataFrame(columns=['UserID'])
        user_data = pd.DataFrame(columns=['CodeUser','UserName','UserCountry','UserProfilePic'])
        #print(temp_temp);
                
        if len(user_device[user_device['DeviceCode'] == device]['IdUser']) == 0:
            user_item = 0;
            user_firstname = '';
            user_lastname = '';
            user_complete_name = user_firstname + user_lastname;
            user_country = '';
            user_profile_pic = '';
            user_diagnosis_code = '';
            diagnosis_list = '';
            
        else:
            user_item = user_device[user_device['DeviceCode'] == device]['IdUser'].item()
            user_firstname = user_df[user_df['Id'] == user_item]['Name'].item();
            user_lastname = user_df[user_df['Id'] == user_item]['LastName'].item();
            user_complete_name = user_firstname +' '+ user_lastname;
            user_country_code = user_df[user_df['Id'] == user_item]['IdCountry'].item();
            user_country = country_df[country_df['Id'] == user_country_code]['Name'].item();
            user_profile_pic = user_df[user_df['Id'] == user_item]['ProfilePicture'].item();
            user_diagnosis_code_list = individual_diagnosis_df[individual_diagnosis_df['IdUser']==user_item]['IdDiagnosis'].tolist()
            user_diagnosis = pd.DataFrame(columns=['Diagnosis'])
            for item in user_diagnosis_code_list: 
                user_diagnosis = user_diagnosis.append({
                    'Diagnosis':diagnosis_df[diagnosis_df['Id']==item]['Name'].item()
                },ignore_index=True) 
                
            diagnosis_list = user_diagnosis.Diagnosis.tolist()
        user_data = user_data.append(
                    {'CodeUser':int(user_item),
                    'UserName':user_complete_name,
                    'UserCountry':user_country,
                    'UserProfilePic':user_profile_pic,
                    'Diagnosis':str(diagnosis_list)},
                    ignore_index=True
                )
        user_data = user_data.to_dict('records')
        #print(user_data.columns)}
        scene_data_buff = pd.DataFrame(columns=['SceneID','SceneType','AnswerType'])
        temp_temp = json.loads(temp_temp)
        for x in temp_temp:            
            temp_df = pd.DataFrame.from_dict(x, orient='index')
            temp_df = temp_df.transpose();
            #print(temp_df['Id'].item())
            scene_data_buff = scene_data_buff.append(
                {'SceneID':temp_df['Id'].item(),
                'CueImage':temp_df['Image'].item(),
                'SceneType':temp_df['SceneType'].item(),
                'AnswerType':temp_df['AnswerType'].item(),
                'CaregiverID':caregiver_id},
                ignore_index=True
            )
        sc_data_buff = (scene_data_buff.groupby(['SceneType','AnswerType']).size().sort_values(ascending=False).reset_index(name='Count').drop_duplicates(subset='AnswerType'))
        sc_data_buff['SceneType'] = sc_data_buff['SceneType'].replace([1],'Static')
        sc_data_buff['SceneType'] = sc_data_buff['SceneType'].replace([2],'Dynamic')
        sc_data_buff['SceneType'] = sc_data_buff['SceneType'].replace([3],'Spoken')
        sc_data_buff['AnswerType'] = sc_data_buff['AnswerType'].replace([1],'Unanswered')
        sc_data_buff['AnswerType'] = sc_data_buff['AnswerType'].replace([2],'Yes/No')
        sc_data_buff['AnswerType'] = sc_data_buff['AnswerType'].replace([3],'Multiple')
        sc_data_buff.rename(columns={'SceneType':'ClassificationType'},inplace=True)
        sc_data_buff.rename(columns={'AnswerType':'Type'},inplace=True)
        scenetype_perdevcue = sc_data_buff[['ClassificationType','Count']]
        scenetype_perdevcue = pd.DataFrame(scenetype_perdevcue.groupby(['ClassificationType'])['ClassificationType'].count())
        scenetype_perdevcue .rename(columns={'ClassificationType':'Count'},inplace=True)
        scenetype_perdevcue .reset_index(inplace=True)
        answertype_perdevcue = sc_data_buff[['Type','Count']]
        answertype_perdevcue = pd.DataFrame(answertype_perdevcue.groupby(['Type'])['Type'].count())
        answertype_perdevcue.rename(columns={'Type':'Count'},inplace=True)
        answertype_perdevcue.reset_index(inplace=True)
        scenetype_perdevcue = scenetype_perdevcue.to_dict(orient = 'records')
        answertype_perdevcue= answertype_perdevcue.to_dict(orient = 'records')
        #print(scenetype_perdevcue)
#        sc_data_buff.rename(columns={'AnswerType':'type'},inplace=True)
#        scenetype_perdevcue  =  pd.DataFrame(scene_data_buff.groupby(['SceneType'])['SceneType'].count())
#        scenetype_perdevcue .rename(columns={'SceneType':'Count'},inplace=True)
#        scenetype_perdevcue .reset_index(inplace=True)
#        #scenetype_perdevcue = scene_data_buff['SceneType'].item()
#        answertype_perdevcue  =  pd.DataFrame(scene_data_buff.groupby(['AnswerType'])['AnswerType'].count())
#        answertype_perdevcue .rename(columns={'AnswerType':'Count'},inplace=True)
#        answertype_perdevcue .reset_index(inplace=True)
#        #answertype_perdevcue = scene_data_buff['AnswerType'].item()
        image = scene_data_buff['CueImage'][0]
#        scene_data = scene_data.append(scene_data_buff,ignore_index=True)
#        #.sort_values(ascending=False) 
#        #.reset_index(name='count') ).to_dict('dict')
#        #answertype_perdevcue =  (scene_data_buff.groupby(['AnswerType']).size()
#        #.sort_values(ascending=False) 
#        #.reset_index(name='count') ).to_dict('dict')
        deliverycue_table_temp = deliverycue_table_temp.append(
            {'CaregiverID':scenes_df['PartitionKey'][index],
            'DevCueID':key,
            'TitleDelivery':title,
            'DeviceID':device,
            'DateDelivery':str(date),
            'ImageDelivery':image,
            'IndividualsData':user_data,
            'ST_total_devcue':str(scenetype_perdevcue),
            'AT_total_devcue':str(answertype_perdevcue)},
            ignore_index=True)
#        deliverycue_table_temp['ST_total_devcue'] = deliverycue_table_temp['ST_total_devcue'].replace([1],'Static')
#        deliverycue_table_temp['ST_total_devcue'] = deliverycue_table_temp['ST_total_devcue'].replace([2],'Dynamic')
#        deliverycue_table_temp['ST_total_devcue'] = deliverycue_table_temp['ST_total_devcue'].replace([3],'Spoken')
#        deliverycue_table_temp['AT_total_devcue'] = deliverycue_table_temp['AT_total_devcue'].replace([1],'Unanswered')
#        deliverycue_table_temp['AT_total_devcue'] = deliverycue_table_temp['AT_total_devcue'].replace([2],'Yes/No')
#        deliverycue_table_temp['AT_total_devcue'] = deliverycue_table_temp['AT_total_devcue'].replace([3],'Multiple')
for index in deliverycue_table_temp.index:
    buff = deliverycue_table_temp['IndividualsData'][index]
    userdata_df_temp = pd.json_normalize(buff[0])
    userdata_by_devcue = userdata_by_devcue.append(userdata_df_temp,ignore_index=True)
DELIVERYCUE_FINAL_TAB = pd.concat([deliverycue_table_temp, userdata_by_devcue], axis=1)
DELIVERYCUE_FINAL_TAB = DELIVERYCUE_FINAL_TAB.drop(columns=['IndividualsData'])
DELIVERYCUE_FINAL_TAB['index_column'] = DELIVERYCUE_FINAL_TAB.index;
DELIVERYCUE_FINAL_TAB.fillna('[]', inplace=True)
sparkDF=spark.createDataFrame(DELIVERYCUE_FINAL_TAB) 
sparkDF.coalesce(1).write.option("header", "true").mode("overwrite").csv("/mnt/datalakegen2/stats/caregiver/CAREGIVER_DEVCUE")
data_location = "/mnt/datalakegen2/stats/caregiver/CAREGIVER_DEVCUE"
files = dbutils.fs.ls(data_location)
csv_file = [x.path for x in files if x.path.endswith(".csv")][0]
dbutils.fs.mv(csv_file, data_location.rstrip('/') + ".csv")
dbutils.fs.rm(data_location, recurse = True)
#
#        
#        
#scene_data_by_cgv =  pd.DataFrame(scene_data.groupby(['CaregiverID']))
#scene_data_by_cgv.rename(columns={0:'CaregiverID',1:'Data'},inplace=True)
#cgv_totals = pd.DataFrame()
#    classf_data.rename(columns={'ClassificationType':'Count'},inplace=True)
#    classf_data.reset_index(inplace=True)
#    typef_data = typef_data.groupby(['Type'])['Type'].count()
#    typef_data.rename(columns={'Type':'Count'},inplace=True)
#    typef_data.reset_index(inplace=True)
    
    
        
#    scenetype_by_cgv =  pd.DataFrame(data.groupby(['SceneType'])['SceneType'].count())
#    scenetype_by_cgv.rename(columns={'SceneType':'Count'},inplace=True)
#    scenetype_by_cgv.reset_index(inplace=True)
#    scenetype_by_cgv['SceneType'] = scenetype_by_cgv['SceneType'].replace([1],'Static')
#    scenetype_by_cgv['SceneType'] = scenetype_by_cgv['SceneType'].replace([2],'Dynamic')
#    scenetype_by_cgv['SceneType'] = scenetype_by_cgv['SceneType'].replace([3],'Spoken')
#    scenetype_by_cgv.rename(columns={'SceneType':'ClassificationType'},inplace=True)
#    #scenetype_by_cgv = scenetype_by_cgv.to_json(orient = 'records')
#    answertype_by_cgv = pd.DataFrame(data.groupby(['AnswerType'])['AnswerType'].count())
#    answertype_by_cgv.rename(columns={'AnswerType':'Count'},inplace=True)
#    answertype_by_cgv.reset_index(inplace=True)
#    answertype_by_cgv['AnswerType'] = answertype_by_cgv['AnswerType'].replace([1],'Unanswered')
#    answertype_by_cgv['AnswerType'] = answertype_by_cgv['AnswerType'].replace([2],'Yes/No')
#    answertype_by_cgv['AnswerType'] = answertype_by_cgv['AnswerType'].replace([3],'Multiple')
#    answertype_by_cgv.rename(columns={'AnswerType':'Type'},inplace=True)
#    type_data = answertype_by_cgv.to_json(orient='records')
#    classification_data = scenetype_by_cgv.to_json(orient='records')
#    #print('********')
#    #print(scenetype_by_cgv)
#    #print(answertype_by_cgv)
#    #print('********')
#    #answertype_by_cgv = answertype_by_cgv.to_json(orient = 'records')
#    #cgv_totals = cgv_totals.append({
#    #    'CaregiverID':cgv,
#    #    'Scenetype_report':scenetype_by_cgv[scenetype_by_cgv['']],
#    #    'Answertype_report':answertype_by_cgv
#    #},ignore_index=True)
#    if 'Static' in scenetype_by_cgv['ClassificationType'].unique(): 
#        static_cue_total = scenetype_by_cgv[scenetype_by_cgv['ClassificationType']=='Static']['Count'].item() 
#    else: 
#        static_cue_total = 0   
#    if 'Spoken' in scenetype_by_cgv['ClassificationType'].unique(): 
#        spoken_cue_total = scenetype_by_cgv[scenetype_by_cgv['ClassificationType']=='Spoken']['Count'].item() 
#    else: 
#        spoken_cue_total = 0 
#    if 'Dynamic' in scenetype_by_cgv['ClassificationType'].unique():
#        dynamic_cue_total = scenetype_by_cgv[scenetype_by_cgv['ClassificationType']=='Dynamic']['Count'].item()
#    else: 
#        dynamic_cue_total = 0
#    if 'Unanswered' in answertype_by_cgv['Type'].unique():
#        unanswered_cue_total = answertype_by_cgv[answertype_by_cgv['Type']=='Unanswered']['Count'].item()
#    else:
#        unanswered_cue_total = 0
#    if 'Yes/No' in answertype_by_cgv['Type'].unique():
#        yes_no_cue_total = answertype_by_cgv[answertype_by_cgv['Type']=='Yes/No']['Count'].item()
#    else:
#        yes_no_cue_total = 0
#    if 'Multiple' in answertype_by_cgv['Type'].unique():
#        multiple_cue_total = answertype_by_cgv[answertype_by_cgv['Type']=='Multiple']['Count'].item()
#    else:
#        multiple_cue_total = 0               
#
#    
#    cgv_totals = cgv_totals.append({
#        'CaregiverID':cgv,
#        'TotalizedClassification':classification_data,
#        'TotalizedType':type_data
#    },ignore_index=True)
#cgv_totals['CaregiverID'] =  [int(item) for item in cgv_totals['CaregiverID']]
for cgvid in cgv_id :
    scenetype_data = deliverycue_table_temp[deliverycue_table_temp['CaregiverID']==cgvid]['ST_total_devcue'].tolist()
    answertype_data = deliverycue_table_temp[deliverycue_table_temp['CaregiverID']==cgvid]['AT_total_devcue'].tolist()
    classf_data = pd.DataFrame()
    typef_data = pd.DataFrame()
    for j in range(len(scenetype_data)):
        tempclass_df = ast.literal_eval(scenetype_data[j])
        tempclass_df = pd.DataFrame(tempclass_df)
        temptype_df = ast.literal_eval(answertype_data[j])
        temptype_df = pd.DataFrame(temptype_df)
        classf_data = classf_data.append(tempclass_df,ignore_index=True)
        typef_data = typef_data.append(temptype_df,ignore_index=True)
    if 'ClassificationType' in classf_data.columns: 
        classf_data = pd.DataFrame(classf_data.groupby(['ClassificationType']).sum()).reset_index()
    if 'Type' in typef_data.columns: 
        typef_data = pd.DataFrame(typef_data.groupby(['Type']).sum()).reset_index()
    user_dates = deliverycue_df[deliverycue_df['PartitionKey'] == cgvid]['Timestamp']  
    devcuetype = pd.DataFrame(deliverycue_splittedbyelement_df[deliverycue_splittedbyelement_df['PartitionKey'] == cgvid]['ElementType'])
    user_info_buff = pd.DataFrame(columns=['Name','UserID','Diagnosis','ProfilePic','Age'])
    rowkey_index = scenes_df[scenes_df['PartitionKey'] == cgvid]['RowKey'].tolist();

        
    if len(devcuetype) == 0: 
        devcuetype = pd.DataFrame(columns=['ElementType','Count']);
    else: 
        devcuetype = pd.DataFrame(devcuetype.groupby(['ElementType'])['ElementType'].count())
        devcuetype.rename(columns = {'ElementType':'Count'}, inplace = True)
        devcuetype.reset_index(inplace=True)
    file_stamp = str(devcuetype.to_json(orient = 'records'))
        
    user_dates = user_dates.sort_values()
    if len(caregiver_patient_df[caregiver_patient_df['CaregiverID'] == cgvid]['Individuals']) == 0: 
        auc = 0; 
    else: 
        auc = caregiver_patient_df[caregiver_patient_df['CaregiverID'] == cgvid]['Individuals'].item()
    if len(user_dates) == 0:
        daily_record = 0; 
        weekly_record = 0; 
        monthly_record = 0;
        user_devcue_count = 0;
        last_devcue_sent = '';
    else:
        daily_record = (user_dates.iloc[-1] - user_dates.iloc[0]).days
        daily_record = int(daily_record)
        weekly_record = daily_record/7
        monthly_record = daily_record/30
        user_devcue_count = caregiver_deliverycue_count_df[caregiver_deliverycue_count_df['UserID'] == cgvid ]['DeliveryCueCount'].item()
        user_devcue_count = int(user_devcue_count)
        last_devcue_sent = user_dates.iloc[-1];
    if daily_record == 0 :
        daily_record = 1;
    if weekly_record == 0 : 
        weekly_record = 1; 
    if monthly_record == 0 : 
        monthly_record = 1;     
    daily_rate = user_devcue_count/ daily_record
    weekly_rate = user_devcue_count/weekly_record
    monthly_rate = user_devcue_count / monthly_record 
    classification_data = classf_data.to_json(orient='records')
    type_data = typef_data.to_json(orient='records')
    caregiver_devcuetimerecord = caregiver_devcuetimerecord.append(
        {'CaregiverID' : cgvid, 
         'DailyRecord' : daily_rate, 
         'WeeklyRecord' :weekly_rate,
         'MonthlyRecord':monthly_rate,
         'HistoricalRecord': user_devcue_count,
         'AssignedUserCount': auc,
         'ElementUsage': file_stamp,
         'LatestCueDate':str(last_devcue_sent),
         'TotalizedClassification':str(classification_data),
         'TotalizedType':str(type_data)},
        ignore_index = True)
    caregiver_devcuetimerecord['index_column'] = caregiver_devcuetimerecord.index;
    
sparkDF=spark.createDataFrame(caregiver_devcuetimerecord) 
sparkDF.coalesce(1).write.option("header", "true").mode("overwrite").csv("/mnt/datalakegen2/stats/caregiver/CAREGIVER")
data_location = "/mnt/datalakegen2/stats/caregiver/CAREGIVER"
files = dbutils.fs.ls(data_location)
csv_file = [x.path for x in files if x.path.endswith(".csv")][0]
dbutils.fs.mv(csv_file, data_location.rstrip('/') + ".csv")
dbutils.fs.rm(data_location, recurse = True)

Out[3]: True

### ORGANIZATION DATASETS GENERATION

In [0]:
organization_report_table = pd.DataFrame(columns = ['Organization_Id','Caregiver_Data','Caregiver_Count','ElementUsage','TotalizedClassification','TotalizedType','DailyRecord','WeeklyRecord','MonthlyRecord','HistoricalRecord'])
user_parent = user_df.groupby(['IdParent'])['IdParent'].count()
user_parent  = pd.DataFrame(user_parent)
user_parent.rename(columns={'IdParent':'ParentCount'},inplace=True)
user_parent.reset_index(inplace=True)
user_parent.rename(columns={'IdParent':'UserId'},inplace=True)
user_parent

for Id in organization_id: 
    if len(user_parent[user_parent['UserId'] == Id]['ParentCount']) == 0: 
        org_parent = 0; 
    else: 
        org_parent = user_parent[user_parent['UserId'] == Id]['ParentCount'].item()
    org_user_list = [];
    org_user_list = user_df[user_df['IdParent']==Id]['Id'].tolist()
    caregiver_data = pd.DataFrame(columns=  ['CaregiverID','DailyRecord','WeeklyRecord','MonthlyRecord','HistoricalRecord','AssignedUserCount','ElementUsage'])
    org_element_usage = pd.DataFrame(columns=['ElementType','Count'])
    org_class = pd.DataFrame()
    org_type = pd.DataFrame()
    org_totals = pd.DataFrame(columns=['DailyRecord','WeeklyRecord','MonthlyRecord','HistoricalRecord'])
    unassigned_users = 0 ; 
    for x in org_user_list: 
        pull_data = pd.DataFrame(caregiver_devcuetimerecord[caregiver_devcuetimerecord['CaregiverID']== x])
        if pull_data['AssignedUserCount'].item() == 0: 
            unassigned_users = unassigned_users + 1
        
        assigned_users = len(org_user_list) - unassigned_users
        daily_record_buff = pull_data['DailyRecord'].item()
        weekly_record_buff = pull_data['WeeklyRecord'].item()
        monthly_record_buff = pull_data['MonthlyRecord'].item()
        historical_record_buff = pull_data['HistoricalRecord'].item()
        org_totals = org_totals.append({'DailyRecord':daily_record_buff,
                                       'WeeklyRecord':weekly_record_buff,
                                       'MonthlyRecord':monthly_record_buff,
                                       'HistoricalRecord':historical_record_buff},ignore_index=True)
        caregiver_elementusage_item = pull_data['ElementUsage'].item()
        caregiver_class_item = pull_data['TotalizedClassification'].item()
        caregiver_type_item = pull_data['TotalizedType'].item()
        if caregiver_elementusage_item == '0': 
            print('caregiver has no history data');
        else:
            caregiver_elementusage_item = pd.read_json(caregiver_elementusage_item, orient='records')
            caregiver_class_item = pd.read_json(caregiver_class_item , orient='records')
            caregiver_type_item = pd.read_json(caregiver_type_item , orient='records')
            org_element_usage = org_element_usage.append(caregiver_elementusage_item,ignore_index=True)
            org_class = org_class.append(caregiver_class_item,ignore_index=True)
            org_type = org_type.append(caregiver_type_item,ignore_index=True)
            print(org_class.columns)
        caregiver_data = caregiver_data.append(pull_data)
    org_element_usage = pd.DataFrame(org_element_usage.groupby(['ElementType']).sum()).reset_index().to_json(orient='records')
    if 'ClassificationType' in org_class.columns:
        org_class = pd.DataFrame(org_class.groupby(['ClassificationType']).sum()).reset_index().to_json(orient='records')
        org_type = pd.DataFrame(org_type.groupby(['Type']).sum()).reset_index().to_json(orient='records')
    else:
        org_class = []
        org_type = []
    
    file_stamp = str(caregiver_data.to_json(orient = 'records'))
    organization_report_table = organization_report_table.append(
    {'Organization_Id' : Id, 
     'Caregiver_Data' : file_stamp, 
     'Caregiver_Count' : org_parent,
     'ElementUsage':str(org_element_usage),
     'TotalizedClassification':str(org_class),
     'TotalizedType':str(org_type),
     'DailyRecord':org_totals['DailyRecord'].mean(),
     'WeeklyRecord':org_totals['WeeklyRecord'].mean(),
     'MonthlyRecord':org_totals['MonthlyRecord'].mean(),
     'HistoricalRecord':org_totals['HistoricalRecord'].sum(),
     'Assigned': assigned_users,
     'NotAssigned':unassigned_users
    },
    ignore_index = True)
    organization_report_table['index_column'] = organization_report_table.index;
    
sparkDF=spark.createDataFrame(organization_report_table) 
sparkDF.coalesce(1).write.option("header", "true").mode("overwrite").csv("/mnt/datalakegen2/stats/organization/ORGANIZATION")
data_location = "/mnt/datalakegen2/stats/organization/ORGANIZATION"
files = dbutils.fs.ls(data_location)
csv_file = [x.path for x in files if x.path.endswith(".csv")][0]
dbutils.fs.mv(csv_file, data_location.rstrip('/') + ".csv")
dbutils.fs.rm(data_location, recurse = True)

Index(['ClassificationType', 'Count'], dtype='object')
Index(['ClassificationType', 'Count'], dtype='object')
Index(['ClassificationType', 'Count'], dtype='object')
Index(['ClassificationType', 'Count'], dtype='object')
Index(['ClassificationType', 'Count'], dtype='object')
Index(['ClassificationType', 'Count'], dtype='object')
Index(['ClassificationType', 'Count'], dtype='object')
Index(['ClassificationType', 'Count'], dtype='object')
Index(['ClassificationType', 'Count'], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], d

### INDIVIDUAL DATASETS GENERATION

In [0]:
individual_id = user_df[user_df['IdRole'] == 2]['Id']
individual_table = pd.DataFrame(columns=['IndividualID','DailyRecord','WeeklyRecord','MonthlyRecord','HistoricalRecord','ElementUsage','LatestDevCueDate'])
individual_devcue_table = pd.DataFrame(columns=['UserId','DeliveryCueList','Classification_total','Type_total']);

for Id in individual_id:
    devices = user_device[user_device['IdUser']==Id]['DeviceCode'].tolist()
    total_class = pd.DataFrame();
    total_type = pd.DataFrame();
    user_deliverycue_history = pd.DataFrame()
    if not devices:
        user_deliverycue_history = pd.DataFrame(columns={'CaregiverID':[''],
            'DevCueID':[''],
            'TitleDelivery':[''],
            'DeviceID':[''],
            'DateDelivery':[''],
            'ImageDelivery':[''],
            'ST_total_devcue':[''],
            'AT_total_devcue':['']
                                                        });
        user_deliverycue_history = user_deliverycue_history.to_dict('records')
        #classf_data={'ClassificationType':['Static','Spoken','Dynamic'],'Count':[0,0,0]}
        #tp_data  = {'Type':['Unanswered','Yes/No','Multiple'],'Count':[0,0,0]}
        #total_class = pd.DataFrame(columns={'Static':[0],
        #                                          'Dynamic':[0],
        #                                          'Spoken':[0]});
        #total_type = pd.DataFrame(columns={
        #    'Unanswered':[0],
        #    'Yes_no':[0],
        #    'Multiple':[0]
        #})
        #type_data = pd.DataFrame(tp_data)
        #classification_data = pd.DataFrame(classf_data)
        #classification_data = classification_data.to_json(orient='records')
        #type_data = type_data.to_json(orient='records')        
    for device in devices: 
        user_delivery_list = DELIVERYCUE_FINAL_TAB[DELIVERYCUE_FINAL_TAB['DeviceID']==device]
        if user_delivery_list.empty: 
            user_deliverycue_history = pd.DataFrame(columns={'CaregiverID':[''],
            'DevCueID':[''],
            'TitleDelivery':[''],
            'DeviceID':[''],
            'DateDelivery':[''],
            'ImageDelivery':[''],
            'ST_total_devcue':[''],
            'AT_total_devcue':['']})
            user_deliverycue_history = user_deliverycue_history.to_dict('records')
            #classf_data={'ClassificationType':['Static','Spoken','Dynamic'],'Count':[0,0,0]}
            #tp_data  = {'Type':['Unanswered','Yes/No','Multiple'],'Count':[0,0,0]}
            #total_class = pd.DataFrame(columns={'Static':[0],
            #                                         'Dynamic':[0],
            #                                          'Spoken':[0]});
            #total_type = pd.DataFrame(columns={
            #    'Unanswered':[0],
            #    'Yes_no':[0],
            #    'Multiple':[0]
            #})
            #type_data = pd.DataFrame(tp_data)
            #classification_data = pd.DataFrame(classf_data)
            #classification_data = classification_data.to_json(orient='records')
            #type_data = type_data.to_json(orient='records')
        else:
            user_deliverycue_history = user_deliverycue_history.append(user_delivery_list)      
            user_deliverycue_history = user_deliverycue_history.to_dict('records')
            scenetotal_data = user_delivery_list['ST_total_devcue'].tolist();
            answertypetotal_data = user_delivery_list['AT_total_devcue'].tolist();
            scenetype_perdev = pd.DataFrame()
            type_perdev = pd.DataFrame()
            for n in range(len(scenetotal_data)):
                tempclass_df = ast.literal_eval(scenetotal_data[n])
                tempclass_df = pd.DataFrame(tempclass_df)
            for m in range(len(answertypetotal_data)): 
                temptype_df = ast.literal_eval(answertypetotal_data[m])
                temptype_df = pd.DataFrame(temptype_df)
                classf_data = classf_data.append(tempclass_df,ignore_index=True)
                typef_data = typef_data.append(temptype_df,ignore_index=True)
            if 'ClassificationType' in classf_data.columns: 
                classf_data = pd.DataFrame(classf_data.groupby(['ClassificationType']).sum()).reset_index()
            if 'Type' in typef_data.columns: 
                typef_data = pd.DataFrame(typef_data.groupby(['Type']).sum()).reset_index()
        total_class = total_class.append(classf_data,ignore_index=True)
        total_type = total_type.append(typef_data,ignore_index=True)
    if 'ClassificationType' in total_class.columns: 
        total_class = pd.DataFrame(total_class.groupby(['ClassificationType']).sum()).reset_index()
    if 'Type' in total_type.columns: 
        total_type = pd.DataFrame(total_type.groupby(['Type']).sum()).reset_index()
                #scenetype_perdev = scenetype_perdev.append({'ClassificationType':scenetotal_data[n]},ignore_index=True)
                #type_perdev = type_perdev.append({'Type':answertypetotal_data[n]},ignore_index=True)
            #scenetype_perdev =  pd.DataFrame(scenetype_perdev.groupby(['SceneType'])['SceneType'].count())
            #scenetype_perdev.rename(columns={'SceneType':'Count'},inplace=True)
            #scenetype_perdev.reset_index(inplace=True)
            #type_perdev =  pd.DataFrame(type_perdev.groupby(['AnswerType'])['AnswerType'].count())
            #type_perdev.rename(columns={'AnswerType':'Count'},inplace=True)
            #type_perdev.reset_index(inplace=True)
#             print('********')
#             print(scenetype_perdev['SceneType'])
#             print(type_perdev['AnswerType'])
#             print('*********')
#             if 'Dynamic' in scenetype_perdev['SceneType'].values: 
#                 dynamic_cue_total = scenetype_perdev[scenetype_perdev['SceneType']=='Dynamic']['Count'].item();
#             else: 
#                 dynamic_cue_total = 0; 
#             if 'Spoken' in scenetype_perdev['SceneType'].values: 
#                 spoken_cue_total = scenetype_perdev[scenetype_perdev['SceneType']=='Spoken']['Count'].item();
#             else: 
#                 spoken_cue_total = 0; 
#             if 'Static' in scenetype_perdev['SceneType'].values:
#                 static_cue_total = scenetype_perdev[scenetype_perdev['SceneType']=='Static']['Count'].item();
#             else:
#                 static_cue_total = 0; 
#             if 'Unanswered' in type_perdev['AnswerType'].values: 
#                 unanswered_cue_total = type_perdev[type_perdev['AnswerType']=='Unanswered']['Count'].item();
#             else: 
#                 unanswered_cue_total = 0; 
#             if 'Yes/No' in type_perdev['AnswerType'].values: 
#                 yes_no_cue_total = type_perdev[type_perdev['AnswerType']=='Yes/No']['Count'].item(); 
#             else: 
#                 yes_no_cue_total = 0; 
#             if 'Multiple' in type_perdev['AnswerType'].values: 
#                 multiple_cue_total = type_perdev[type_perdev['AnswerType']=='Multiple']['Count'].item(); 
#             else: 
#                 multiple_cue_total = 0;
#             classf_data={'ClassificationType':['Static','Spoken','Dynamic'],'Count':[static_cue_total,spoken_cue_total,dynamic_cue_total]}
#             tp_data  = {'Type':['Unanswered','Yes/No','Multiple'],'Count':[unanswered_cue_total,yes_no_cue_total,multiple_cue_total]}
#             total_class = pd.DataFrame(classf_data)
#             total_type = pd.DataFrame(tp_data)
    classification_data = total_class.to_json(orient='records')
    type_data = total_type.to_json(orient='records')
        #type_data = type_data.to_json(orient='records')
    individual_devcue_table = individual_devcue_table.append({
        'UserId':Id,
        'DeliveryCueList':str(user_deliverycue_history),
        'Classification_total':str(classification_data),
        'Type_total':str(type_data)
         },ignore_index=True)
INDIVIDUAL_DEVCUE_TAB = pd.DataFrame()
for index in individual_devcue_table.index:
    data = ast.literal_eval(individual_devcue_table['DeliveryCueList'][index])
    Id = individual_devcue_table['UserId'][index]
    if not data:
        print()
    else:
        master_copy = individual_devcue_table[individual_devcue_table['UserId']==Id]
        userdata_by_devcue = pd.DataFrame();
        copy_table = pd.DataFrame()
        for i in range(len(data)):
            copy_table = copy_table.append(master_copy,ignore_index=True)
            row = pd.json_normalize(data[i])
            userdata_by_devcue = userdata_by_devcue.append(row,ignore_index=True)
            #temp_df = pd.DataFrame(data)
        deliverytable_by_user = pd. concat([copy_table, userdata_by_devcue], axis=1)
        INDIVIDUAL_DEVCUE_TAB = INDIVIDUAL_DEVCUE_TAB.append(deliverytable_by_user,ignore_index=True)
INDIVIDUAL_DEVCUE_TAB['index_column'] = INDIVIDUAL_DEVCUE_TAB.index;
sparkDF=spark.createDataFrame(INDIVIDUAL_DEVCUE_TAB) 
sparkDF.coalesce(1).write.option("header", "true").mode("overwrite").csv("/mnt/datalakegen2/stats/individual/INDIVIDUAL_DEVCUE")
data_location = "/mnt/datalakegen2/stats/individual/INDIVIDUAL_DEVCUE"
files = dbutils.fs.ls(data_location)
csv_file = [x.path for x in files if x.path.endswith(".csv")][0]
dbutils.fs.mv(csv_file, data_location.rstrip('/') + ".csv")
dbutils.fs.rm(data_location, recurse = True)
  

for Id in individual_id:
    individual_devcue_id_list = INDIVIDUAL_DEVCUE_TAB[INDIVIDUAL_DEVCUE_TAB['CodeUser']==Id]['DevCueID'].tolist()
    ind_devcue_count = len(individual_devcue_id_list)
    user_dates = INDIVIDUAL_DEVCUE_TAB[INDIVIDUAL_DEVCUE_TAB['CodeUser'] == Id]['DateDelivery']  
    devcuetype = pd.DataFrame(columns=['ElementType','Count']);
    for Ix in individual_devcue_id_list:
        devcuetype_element =deliverycue_splittedbyelement_df[deliverycue_splittedbyelement_df['RowKey'] == Ix]['ElementType'].tolist()
        for item in devcuetype_element:
            devcuetype = devcuetype.append({'ElementType':item},ignore_index=True)
        
        #print(devcuetype_element)
    devcuetype = pd.DataFrame(devcuetype.groupby(['ElementType'])['ElementType'].count())
    devcuetype.rename(columns = {'ElementType':'Count'}, inplace = True)
    devcuetype.reset_index(inplace=True)         
    file_stamp = str(devcuetype.to_json(orient = 'records'))
    user_dates = user_dates.sort_values()
    if len(caregiver_patient_df[caregiver_patient_df['CaregiverID'] == Id]['Individuals']) == 0: 
        auc = 0; 
    else: 
        auc = caregiver_patient_df[caregiver_patient_df['CaregiverID'] == Id]['Individuals'].item()
    if len(user_dates) == 0:
        daily_record = 0; 
        weekly_record = 0; 
        monthly_record = 0;
        user_devcue_count = 0;
        last_devcue_sent = '';
    else:
        last =datetime.strptime(user_dates.iloc[-1], '%Y-%m-%d %H:%M:%S.%f')
        first = datetime.strptime(user_dates.iloc[0], '%Y-%m-%d %H:%M:%S.%f')
        daily_record = ( last-first ).days
        daily_record = int(daily_record)
        weekly_record = daily_record/7
        monthly_record = daily_record/30
        user_devcue_count = len(individual_devcue_id_list )
        user_devcue_count = int(user_devcue_count)
        last_devcue_sent = user_dates.iloc[-1];
    if daily_record == 0 :
        daily_record = 1;
    if weekly_record == 0 : 
        weekly_record = 1; 
    if monthly_record == 0 : 
        monthly_record = 1;     
    daily_rate = user_devcue_count/ daily_record
    weekly_rate = user_devcue_count/weekly_record
    monthly_rate = user_devcue_count / monthly_record
    classification_total = individual_devcue_table[individual_devcue_table['UserId']==Id]['Classification_total'].item()
    type_total = individual_devcue_table[individual_devcue_table['UserId']==Id]['Type_total'].item()
    
    individual_table = individual_table.append(
        {'IndividualID' : Id, 
         'DailyRecord' : daily_rate, 
         'WeeklyRecord' :weekly_rate,
         'MonthlyRecord':monthly_rate,
        'HistoricalRecord': user_devcue_count,
        'ElementUsage': file_stamp,
        'LatestDevCueDate':last_devcue_sent,
        'TotalizedClassification':classification_total,
        'TotalizedType':type_total},
        ignore_index = True)
    individual_table['index_column'] = individual_table.index;
sparkDF=spark.createDataFrame(individual_table) 
sparkDF.coalesce(1).write.option("header", "true").mode("overwrite").csv("/mnt/datalakegen2/stats/individual/INDIVIDUAL")
data_location = "/mnt/datalakegen2/stats/individual/INDIVIDUAL"
files = dbutils.fs.ls(data_location)
csv_file = [x.path for x in files if x.path.endswith(".csv")][0]
dbutils.fs.mv(csv_file, data_location.rstrip('/') + ".csv")
dbutils.fs.rm(data_location, recurse = True)























































































Out[29]: True

In [0]:
individual_table

,IndividualID,DailyRecord,WeeklyRecord,MonthlyRecord,HistoricalRecord,ElementUsage,LatestDevCueDate,TotalizedClassification,TotalizedType,index_column
0,272,0.0,0.0,0.0,0,[],,[],[],0
1,278,0.0,0.0,0.0,0,[],,[],[],1
2,280,0.0,0.0,0.0,0,[],,[],[],2
3,281,0.0,0.0,0.0,0,[],,[],[],3
4,285,0.0,0.0,0.0,0,[],,[],[],4
...,...,...,...,...,...,...,...,...,...,...
86,447,0.0,0.0,0.0,0,[],,[],[],86
87,448,0.0,0.0,0.0,0,[],,[],[],87
88,449,0.0,0.0,0.0,0,[],,[],[],88
89,450,0.0,0.0,0.0,0,[],,[],[],89


### ADMIN DATASETS GENERATION